<a href="https://colab.research.google.com/github/Phillyprogramer215/DS-Unit-1-Sprint-2-Statistics/blob/master/Copy_of_DS_Unit_1_Sprint_Challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 2

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 is a variable that indicates whether or not a person's request for credit has been approved or denied. This is a good candidate for a y variable since we might want to use the other features to predict this one. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
# TODO
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
import seaborn as sns

In [0]:
df = pd.read_csv('crx.data', header=None, na_values='?')

In [0]:
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
210,b,39.33,5.875,u,g,cc,h,10.00,t,t,14,t,g,399.0,0,+
301,b,18.25,0.165,u,g,d,v,0.25,f,f,0,t,s,280.0,0,-
200,a,24.08,0.500,u,g,q,h,1.25,t,t,1,f,g,0.0,678,+
584,a,28.08,15.000,y,p,e,z,0.00,t,f,0,f,g,0.0,13212,+
276,a,19.17,5.415,u,g,i,h,0.29,f,f,0,f,g,80.0,484,-


In [0]:
df.isnull().sum().sort_values(ascending=True)

2      0
7      0
8      0
9      0
10     0
11     0
12     0
14     0
15     0
3      6
4      6
5      9
6      9
0     12
1     12
13    13
dtype: int64

In [0]:
df.dtypes

0      object
1     float64
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12     object
13    float64
14      int64
15     object
dtype: object

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
# TODO
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [0]:
df[15].value_counts()

-    383
+    307
Name: 15, dtype: int64

In [0]:
first_condition = df[15]== '-'
second_condition = df[15]== '+'

In [0]:
minus_df = df[first_condition]
plus_df = df[second_condition]

In [0]:
minus_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
70,b,32.33,7.5,u,g,e,bb,1.585,t,f,0,t,s,420.0,0,-
71,b,34.83,4.0,u,g,d,bb,12.500,t,f,0,t,g,NaN,0,-
72,a,38.58,5.0,u,g,cc,v,13.500,t,f,0,t,g,980.0,0,-
73,b,44.25,0.5,u,g,m,v,10.750,t,f,0,f,s,400.0,0,-
74,b,44.83,7.0,y,p,c,v,1.625,f,f,0,f,g,160.0,2,-


In [0]:
plus_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
590,b,30.17,6.50,u,g,cc,v,3.125,t,t,8,f,g,330.0,1200,+
20,b,25.00,11.25,u,g,c,v,2.500,t,t,17,f,g,200.0,1208,+
183,a,57.08,19.50,u,g,c,v,5.500,t,t,7,f,g,0.0,3000,+
490,a,47.25,0.75,u,g,q,h,2.750,t,t,1,f,g,333.0,892,+
510,b,13.75,4.00,y,p,w,v,1.750,t,t,2,t,g,120.0,1000,+


In [0]:
cd1 = pd.crosstab(plus_df[0], plus_df[15])
cd1

15,+
0,
a,98
b,206


In [0]:
plus_df.isnull().sum()

0     3
1     2
2     0
3     4
4     4
5     4
6     4
7     0
8     0
9     0
10    0
11    0
12    0
13    6
14    0
15    0
dtype: int64

In [0]:
ttest_ind(plus_df[13], minus_df[13], nan_policy='omit')

Ttest_indResult(statistic=-2.6358251986645476, pvalue=0.008586135473979569)

In [0]:
plus_df[13].mean()

164.421926910299

In [0]:
minus_df[13].mean()

199.6994680851064

In [0]:
print(plus_df[13].mean())
print(minus_df[13].mean())

164.421926910299
199.6994680851064


In [0]:
ttest_ind(plus_df[2], minus_df[2], nan_policy='omit')

Ttest_indResult(statistic=5.52998337614816, pvalue=4.551680702308068e-08)

In [0]:
print(plus_df[2].mean())
print(minus_df[2].mean())

5.904951140065144
3.839947780678851


In [0]:
contingency_table = pd.crosstab(df[15], df[0])
contingency_table

,__dummy__,__dummy__
15,,
+,98,206
-,112,262


In [0]:
plusdf = contingency_table.iloc[0][0:2].values
plusdf

array([ 98, 206])

In [0]:
minusdf = contingency_table.iloc[1][0:2].values
minusdf

array([112, 262])

In [0]:
row_sums = contingency_table.iloc[0:2, 1].values
col_sums = contingency_table.iloc[0:2, 0:2].values

In [0]:
totalplus = plusdf[0] + plusdf[1]
totalplus


304

In [0]:
totalminus = minusdf[0] + minusdf[1]
totalminus

374

In [0]:
total = totalplus + totalminus
total

678

In [0]:
expected = []
for i in range(len(row_sums)):
    expected_row = []
    for column in col_sums:
        expected_val = column*row_sums[i]/total
        expected_row.append(expected_val)
    expected.append(expected_row)

expected = np.array(expected)
print(expected.shape)  
print(expected)

(2, 2, 2)
[[[ 29.77581121  62.5899705 ]
  [ 34.02949853  79.60471976]]

 [[ 37.87020649  79.60471976]
  [ 43.28023599 101.24483776]]]


In [0]:
observed1 = pd.crosstab(plus_df[15], plus_df[0]).values
observed1

array([[ 98, 206]])

In [0]:
#test for the plus dataframes
chi_squared = ((observed1 - expected)**2/(expected)).sum() 
print(f"Chi-Squared: {chi_squared}")

Chi-Squared: 1279.58543926933


In [0]:
observed2 = pd.crosstab(minus_df[15], minus_df[0]).values
observed2

array([[112, 262]])

In [0]:
#test for the minus dataframes
chi_squared = ((observed2 - expected)**2/(expected)).sum() 
print(f"Chi-Squared: {chi_squared}")

Chi-Squared: 2386.3182074272236


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

*Your words here!*

1. the two t-tests that i ran was the difference between the plus df and the minus df

2. the two chi-squared tests that i ran was based on the same information


3. in my opinion the most challenging part was having to review on everything that we learned up until this point, when preparing for the sprint i went over most of the statistics notes 
but i didn't review any of the past lectures from the previous sprint. therefore i had to go back and review those notes as well as the notes from this sprint


> Indented block



# Stretch Goals: 
Do these to get a 3. These are not required in order to pass the Sprint Challenge.

## Part 1: 

Make sure that all of your dataframe columns have the appropriate data types. *Hint:* If a column has the datatype of "object" even though it's made up of float or integer values, you can coerce it to act as a numeric column by using the `pd.to_numeric()` function. In order to get a 3 on this section make sure that your data exploration is particularly well commented, easy to follow, and thorough.

## Part 2:

Write functions that can calculate t-tests and chi^2 tests on all of the appropriate column combinations from the dataset. (Remember that certain tests require certain variable types.)

## Part 3: 

Calculate and report confidence intervals on your most important mean estimates (choose at least two). Make some kind of a graphic or visualization to help us see visually how precise these estimates are.

In [0]:
# You can work the stretch goals down here or back up in their regular sections
# just make sure that they are labeled so that we can easily differentiate
# your main work from the stretch goals.